In [1]:
import pymongo as pdb

import libs.tor_session as ts
import libs.mat_definitions as md
import libs.async_native as asn



import requests as req
from datetime import datetime
from queue import SimpleQueue



In [2]:
client = pdb.MongoClient('localhost',27017)

In [3]:
idex_pages = md.list_of_pages()

In [4]:
tor_session,process = ts.make_session()

started process.
Trying: C:\Program Files\Tor\tor.exe
Trying: C:\Program Files (x86)\Tor\tor.exe
Trying: C:\Program Files\Vidalia Bundle\Tor\tor.exe
Trying: C:\Program Files (x86)\Vidalia Bundle\Tor\tor.exe
Trying: C:\Users\trist\Desktop\Tor Browser\Browser\TorBrowser\Tor\tor.exe
Trying: D:\Dropbox\Digital Projects\Webscrapers\matweb\bin\tor-nt\tor.exe
Tor linked. Confirming connectivity.

LOCAL IP IS:
{
  "origin": "98.192.224.11"
}
 



Configuring socks...
REMOTE IP IS:
200
{
  "origin": "185.220.101.134"
}



In [6]:
def dict_to_mongodb(entries, collection, key_name, value_name, **other_feilds):
    new_data = [ {key_name:key.to_bytes(16,"big"),value_name:value,**other_feilds} 
                 for key, value in entries.items()]
    return collection.insert_many(new_data)
    

In [7]:
def fill_matIDs_matweb(urls, session, target_collection, batch_args=dict()):
    MessageQue = SimpleQueue()
    fs_gen = asn.exec_func_stack(urls,
                         (asn.message_passthrough, run_get, md.pageprocess),
                         (("Fetching URL:{val}",),(session,),()),
                        )
    asn.nb_queue_print(MessageQue,500,0.5)
    res = asn.batch_concurrent(fs_gen, **batch_args)
    #asn.queue_print(MessageQue,10)
    print(f"Retreived {len(res)} pages, {sum([len(i) for i in res])} total entries")
    responses = [dict_to_mongodb(e,target_collection,"MatID","Name",
                                 site="www.matweb.com", retreived=datetime.now())
                 for e in res]
    print(f"{len(responses)} pages added to collection {target_collection.full_name}")
    for response, url in zip(responses,urls):
        print(f"> {len(response.inserted_ids)} matIDs were inserted from page {url}")
    return responses

    
    

    

In [8]:
fill_matIDs_matweb(idex_pages,tor_session,client.materials.matID,{'workers':10,'batch_size':40,'pause':10})

Fetching URL:http://www.matweb.com/search/GetAllMatls.aspx?p=1Fetching URL:http://www.matweb.com/search/GetAllMatls.aspx?p=2

Fetching URL:http://www.matweb.com/search/GetAllMatls.aspx?p=3
Fetching URL:http://www.matweb.com/search/GetAllMatls.aspx?p=4
Fetching URL:http://www.matweb.com/search/GetAllMatls.aspx?p=5
Fetching URL:http://www.matweb.com/search/GetAllMatls.aspx?p=6
Fetching URL:http://www.matweb.com/search/GetAllMatls.aspx?p=7
Fetching URL:http://www.matweb.com/search/GetAllMatls.aspx?p=8
Fetching URL:http://www.matweb.com/search/GetAllMatls.aspx?p=9
Fetching URL:http://www.matweb.com/search/GetAllMatls.aspx?p=10
Fetching URL:http://www.matweb.com/search/GetAllMatls.aspx?p=11
Fetching URL:http://www.matweb.com/search/GetAllMatls.aspx?p=12
Fetching URL:http://www.matweb.com/search/GetAllMatls.aspx?p=13
Fetching URL:http://www.matweb.com/search/GetAllMatls.aspx?p=14
Fetching URL:http://www.matweb.com/search/GetAllMatls.aspx?p=15
Fetching URL:http://www.matweb.com/search/GetAllM